## aMapReduce Framework

Agentics enable scalable execution of LLM workflows by implementing a MapReduce framework which enable the async use of LLM blended with regular python code.

### Amap

Amap functions enable async execution of functions over all the states of an AG.
In its current implementation, Agentics supports 1:1 maps that maps all states of an AG into states of the same type.

In [1]:
from agentics import Agentics as AG
from typing import Optional
from pydantic import BaseModel

## Define the data model for stock market data
class StockMarketState(BaseModel):   
    Date: Optional[str] = None
    Open: Optional[float] = None
    High: Optional[float] = None
    Low: Optional[float] = None
    Close: Optional[float]  = None
    Volume: Optional[int] = None
    Adj_Close: Optional[float] = None
    volatility: Optional[float] = None
    News: Optional[str] = None
    Explanation: Optional[str] = None
    
## import the data
dj_data = AG.from_csv("data/dow_jones.csv", atype=StockMarketState)

## Define the function to compute daily variation percentage
## The input and output are both StockMarketState objects
## note the use of async
async def get_daily_variation_percentage(state: StockMarketState) -> StockMarketState:
    state.volatility = (float(state.High) - float(state.Low)) / float(state.Low) * 100
    return state

## Apply the function to all states using amap
dj_data = await dj_data.amap(get_daily_variation_percentage)

print(dj_data.pretty_print())

2025-08-31 14:27:32.011 | DEBUG    | agentics.core.llm_connections:<module>:66 - AGENTICS is connecting to the following LLM API providers:
2025-08-31 14:27:32.012 | DEBUG    | agentics.core.llm_connections:<module>:69 - 0 - WatsonX
2025-08-31 14:27:32.012 | DEBUG    | agentics.core.llm_connections:<module>:74 - 1 - Gemini
2025-08-31 14:27:32.012 | DEBUG    | agentics.core.llm_connections:<module>:78 - 2 - OpenAI
2025-08-31 14:27:32.012 | DEBUG    | agentics.core.llm_connections:<module>:80 - Please add API keys in .env file to add or disconnect providers.
2025-08-31 14:27:32.022 | DEBUG    | agentics.core.agentics:from_csv:313 - Importing Agentics of type StockMarketState from CSV data/dow_jones.csv
2025-08-31 14:27:32.028 | DEBUG    | agentics.core.agentics:amap:207 - Executing amap on function <function get_daily_variation_percentage at 0x107d04f40>
2025-08-31 14:27:32.029 | DEBUG    | agentics.core.agentics:amap:232 - 20 states processed. 3.8504600524902344e-05 seconds average per 

Atype : <class '__main__.StockMarketState'>
Date: '2016-07-01'
Open: 17924.240234
High: 18002.380859
Low: 17916.910156
Close: 17949.369141
Volume: 82160000
Adj_Close: null
volatility: 0.47703930117312493
News: null
Explanation: null

Date: '2016-06-30'
Open: 17712.759766
High: 17930.609375
Low: 17711.800781
Close: 17929.990234
Volume: 133030000
Adj_Close: null
volatility: 1.2353831025172834
News: null
Explanation: null

Date: '2016-06-29'
Open: 17456.019531
High: 17704.509766
Low: 17456.019531
Close: 17694.679688
Volume: 106380000
Adj_Close: null
volatility: 1.423521751672572
News: null
Explanation: null

Date: '2016-06-28'
Open: 17190.509766
High: 17409.720703
Low: 17190.509766
Close: 17409.720703
Volume: 112190000
Adj_Close: null
volatility: 1.2751857855522295
News: null
Explanation: null

Date: '2016-06-27'
Open: 17355.210938
High: 17355.210938
Low: 17063.080078
Close: 17140.240234
Volume: 138740000
Adj_Close: null
volatility: 1.712064050948546
News: null
Explanation: null

Date: '2

aReduce Enable executing fuctions on the whole list of elements, for example to get the top 10 days in which there was highest volatility in the market

In [ ]:

from ddgs import DDGS

## Define a function to get news for a given date using the DDGS tool
async def get_news(state):
    state.News=str(DDGS().text(f"What happended to the stock market and dow jones on {state.Date}", max_results=10))
    return state    

## Define a reduce function to get the top 10 days with highest volatility
## note that the input is the list of all StockMarketState objects as opposed to a single object in the map function

async def get_highest_volatility_days(states:list[StockMarketState]) -> list[StockMarketState]:
    
    # sort the states by volatility and return the top 10
    top_states = AG(atype=StockMarketState,
                    states= sorted(states, 
                                   key=lambda x: abs(x.volatility) if x.volatility is not None else 0, 
                                   reverse=True)[:10])
                                   
    top_states= await top_states.amap(get_news)
    top_states.instructions = "Explain the market volatility given the news"
    top_states = await top_states.self_transduction(
    ["Date", "Open", "High", "Low", "Close", "Volume", "volatility", "News"],["Explanation"])
    return top_states.states

results = await dj_data.areduce(get_highest_volatility_days)



print(dj_data.pretty_print())

/var/folders/bq/j30lnk112c9dqh0vqvpzqqbr0000gn/T/ipykernel_81433/2289132463.py:18: RuntimeWarning: coroutine 'Agentics.amap' was never awaited
  top_states.amap(get_news)


TypeError: 'NoneType' object is not iterable

In [ ]:



highest_volatility_days = await highest_volatility_days.amap(get_news)
highest_volatility_days.instructions = "Explain the market volatility given the news"
explanations = await highest_volatility_days.self_transduction(
    ["Date", "Open", "High", "Low", "Close", "Volume", "volatility", "News"],["Explanation"]
)
print(explanations.pretty_print())

2025-08-31 14:19:44.553 | DEBUG    | agentics.core.agentics:amap:207 - Executing amap on function <function get_news at 0x12c889ee0>
2025-08-31 14:19:57.900 | DEBUG    | agentics.core.agentics:amap:232 - 10 states processed. 1.3346190929412842 seconds average per state in the last chunk ...
2025-08-31 14:19:57.903 | DEBUG    | agentics.core.agentics:__lshift__:519 - Executing task: Explain the market volatility given the news
10 states will be transduced
2025-08-31 14:19:57.904 | DEBUG    | agentics.core.agentics:__lshift__:613 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-08-31 14:20:05.935 | DEBUG    | agentics.core.agentics:__lshift__:649 - Processed 10 states in 8.030160903930664 seconds
2025-08-31 14:20:05.936 | DEBUG    | agentics.core.agentics:__lshift__:701 - 10 states processed in 0.4015080451965332 seconds average per state ...


Atype : <class '__main__.StockMarketState'>
Date: '2008-10-10'
Open: 8568.669922
High: 8901.280273
Low: 7882.509766
Close: 8451.19043
Volume: 674920000
Adj_Close: null
volatility: 12.924443321266926
News: "[{'title': 'Global financial crisis in October 2008 - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Global_financial_crisis_in_October_2008',\
  \ 'body': 'On October 10 , within the first five minutes of the trading session\
  \ on Wall Street, the Dow Jones Industrial Average plunged 697 points, falling below\
  \ 7900 to its lowest level since March 17, 2003.'}, {'title': '2008 financial crisis',\
  \ 'href': 'https://en.wikipedia.org/wiki/2008_financial_crisis', 'body': 'October\
  \ 6\u201310, 2008: From October 6\u201310, 2008, the Dow Jones Industrial Average\
  \ (DJIA) closed lower in all five sessions. Volume levels were record-breaking.\
  \ The ...'}, {'title': 'Panicky Sellers Darken Afternoon on Wall Street - The New\
  \ ...', 'href': 'https://www.nytimes.com/2008